# Lab 4 : Keltner Channel

![](images/05.png)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
#!pip install yfinance
import yfinance as yf

### 1. 網路爬蟲抓股價

In [ ]:
STOCK_CODE='2330.TW'
df_TSMC = yf.Ticker(STOCK_CODE).history(period="6mo")


### 2. Keltner Channel

In [ ]:
df_TSMC.head()

In [ ]:
# 计算 Typical Price
df_TSMC['typical_price'] = (df_TSMC['High'] + df_TSMC['Low'] + df_TSMC['Close']) / 3

# 计算中轨（20日移动平均）
n = 20
df_TSMC['middle_band'] = df_TSMC['typical_price'].rolling(window=n).mean()

# 计算 ATR（14日的平均真实范围）
n_atr = 14
high_prices = df_TSMC['High']
low_prices = df_TSMC['Low']
previous_closes = df_TSMC['Close'].shift(1)
true_ranges = pd.Series([max(high_prices[i] - low_prices[i], abs(high_prices[i] - previous_closes[i]), abs(low_prices[i] - previous_closes[i])) for i in range(len(df_TSMC))])
df_TSMC['atr'] = true_ranges.ewm(span=n_atr, min_periods=n_atr).mean().values

# 计算上轨和下轨
multiplier = 2  # 可根据需要调整
df_TSMC['upper_band'] = df_TSMC['middle_band'] + multiplier * df_TSMC['atr']
df_TSMC['lower_band'] = df_TSMC['middle_band'] - multiplier * df_TSMC['atr']


In [ ]:
plt.figure(figsize=(22,3))
plt.plot(df_TSMC['middle_band'], label='MA20')
plt.plot(df_TSMC['Close'], label='TSMC_Close')
plt.plot(df_TSMC['Close'], 'o')
plt.fill_between(df_TSMC.index, df_TSMC['lower_band'], df_TSMC['upper_band'], alpha=0.2, label='Keltner Channel')
plt.xticks(rotation=40, ha='right');
plt.legend()
plt.grid()
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=3))  # 设置xticks label
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))  # 设置日期格式
plt.show()


### 3. 交易策略
- 當均線斜率往上 價格由下往上穿過上/下通道 買進多單
- 當均線斜率往下 價格由上往下跌破下/下通道 買進空單
